In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bts
import requests as req
import os

In [2]:
with open('html/intro_naves.html','r',encoding='UTF-8') as file:
    contenido=file.read()
soup=bts(contenido,'html.parser')
href=[a.get('href').replace('..','https://space.skyrocket.de') for a in soup.find_all('table')[0].find_all('a') if int(a.text) > 1950 and int(a.text) < 2024]

In [3]:
for i in range(len(href)):
    pag=req.get(href[i])
    pag_html=pag.text
    if not os.path.exists(f'html/pagina{i}.html'):
        with open(f'html/pagina{i}.html', 'w', encoding='utf-8') as archivo:
            archivo.write(pag_html)

In [56]:
with open('sat/pagina0.html','r',encoding='UTF-8') as file:
    contenido = file.read()

soup = bts(contenido, 'html.parser')

dictio = {e.find('th').text: [] for e in soup.find_all('table')[0].find_all('tr')}
dictio['sat'] = []

In [57]:
def dfs(n_pagina,archivo):
    
    if archivo=='html':

        with open(f'{archivo}/pagina{n_pagina}.html','r',encoding='UTF-8') as file:
            contenido=file.read()

        soup = bts(contenido,'html.parser')
        soup.find('table')

        satelites=[]
        satelites=[soup.find('table').find_all('a')[i].get('href').replace('..','https://space.skyrocket.de') for i in range(0,len(soup.find('table').find_all('a')),2)]

        elementos = [i.text.replace('\n','') for i in soup.find('table').find_all('td')]
        claves = [i.text for i in soup.find('table').find('tr') if i.text!='\n']
        values = [elementos[i:i+6] for i in range(0,len(elementos),6)]

        ref = soup.find('div', {'class': 'container'}).text.split('\n')[soup.find('div', {'class': 'container'}).text.split('\n').index('Launch sites:'):soup.find('div', {'class': 'container'}).text.split('\n').index('\t')-2]
        colum_ref = {e[0]:e[1] for e in [[e.strip() for e in i] for i in [i.split('=') for i in ref[1:]]]}
        colum_ref

        diccio = {i: [] for i in claves}

        for sublist in values:
            for a in range(6):
                diccio[claves[a]].append(sublist[a])
                
        df = pd.DataFrame(diccio)
        comp = [e.split() for e in df['Site'].to_list()]
        df['Site'] = df['Site'].apply(lambda x : x.split()[-1])
        df['Site_ref'] = [e[0] for e in comp]
        df['site_info'] = df['Site_ref'].map(colum_ref)
        return df,satelites
            
    elif archivo=='sat':
        global dictio
        with open(f'sat/pagina{n_pagina}.html','r',encoding='UTF-8') as file:
            contenido = file.read()

        soup = bts(contenido, 'html.parser')
        i={e:[]for e in dictio.keys()}
        i['sat'] = [soup.find('h1').text]
        
        for e in soup.find('table',{'class':'data'}).find_all('tr'):
            
            if e.find('th').text in list(dictio.keys()):
                
                try:
                    if e.find('td').text=='':
                        i[e.find('th').text]='-'
                    else:
                        i[e.find('th').text]=e.find('td').text
                    df=pd.DataFrame(i)
                    return df
                except:
                    continue
            else:
                continue
    '''
    n_pagina representa a la pagina que deseas convertir en df dentro de una carpeta especifica que se encuentre en tu archivo html
    convierte todo el contenido de la pagina en n df donde se ve la informacion del contenido.
    solo es aplicado a este tipo de archivo

    html
    devuelve un df 
    devuelve una lista de links sobre los satelites

    sat
    devuelve un df
    '''

In [10]:
df_proyectos = pd.DataFrame()
links_satelites=[]
for i in range(66):
    df_actual,satelites = dfs(i,'html')
    df_proyectos = pd.concat([df_proyectos, df_actual], axis=0).reset_index(drop=True)
    links_satelites+=satelites

In [63]:
num=len(links_satelites)

In [46]:
for a in range(100):
    with open(f'sat/pagina{a}.html','r',encoding='UTF-8') as file:
        contenido = file.read()

    soup = bts(contenido, 'html.parser')
    i=dictio
    i['sat'] = [soup.find('h1').text]
    try:
        for e in soup.find('table',{'class':'data'}).find_all('tr'):
            
            if e.find('th').text in list(dictio.keys()):
                
                try:
                    if e.find('td').text=='':
                        i[e.find('th').text]='-'
                    else:
                        i[e.find('th').text]=e.find('td').text
                    print(pd.DataFrame(i))
                except:
                    continue
            else:
                continue
    except Exception as e:
        print(f"Error en la página {a}: {str(e)}")

  Nation:                     Type / Application:            Operator:   
0    USSR  Reconnaissance, photo (film scan type)  US Air Force (USAF)  \

  Contractors:                     Equipment: Configuration: Propulsion:   
0            -  E-1 camera, F-1 ELINT package        Agena-A   Bell 8048  \

      Power: Lifetime: Mass:                  Orbit:                  sat  
0  Batteries   10 days     -  557 km × 474 km, 97.4°  Sputnik 1 (PS-1 #1)  
  Nation: Type / Application:            Operator: Contractors:   
0    USSR          Technology  US Air Force (USAF)            -  \

                      Equipment: Configuration: Propulsion:     Power:   
0  E-1 camera, F-1 ELINT package        Agena-A   Bell 8048  Batteries  \

  Lifetime: Mass:                  Orbit:                  sat  
0   10 days     -  557 km × 474 km, 97.4°  Sputnik 1 (PS-1 #1)  
  Nation: Type / Application: Operator: Contractors:   
0    USSR          Technology         -            -  \

                   

In [40]:
for i in range(num):
    if not os.path.exists(f'sat/pagina{i}.html'):
        pag=req.get(links_satelites[i])
        pag_html=pag.text
        with open(f'sat/pagina{i}.html', 'w', encoding='utf-8') as archivo:
            archivo.write(pag_html)

In [66]:
df_proye = pd.DataFrame()
for i in range(num):
    try:
        df_proye_sat = dfs(i,'sat')
        df_proye = pd.concat([df_proye, df_proye_sat], axis=0).reset_index(drop=True)
    except Exception as e:
        print(f'{i}:{e}')

214:'NoneType' object has no attribute 'find_all'
346:'NoneType' object has no attribute 'find_all'
580:'NoneType' object has no attribute 'find_all'
666:'NoneType' object has no attribute 'find_all'
1210:'NoneType' object has no attribute 'find_all'
1325:'NoneType' object has no attribute 'find_all'
2501:'NoneType' object has no attribute 'text'
3002:'NoneType' object has no attribute 'text'
4265:'NoneType' object has no attribute 'text'
4428:'NoneType' object has no attribute 'text'
4493:'NoneType' object has no attribute 'text'
5580:'NoneType' object has no attribute 'text'
5836:'NoneType' object has no attribute 'text'
8593:'NoneType' object has no attribute 'text'
9693:'NoneType' object has no attribute 'text'
10269:'NoneType' object has no attribute 'text'


In [26]:
df_proyectos

,ID,Date,Payload(s),Launch Vehicle,Site,Remark,Site_ref,site_info
0,1957 α (001),04.10.1957,Sputnik 1 (PS-1 #1),Sputnik (1),LC-1/5,,Ba,"Baikonur (Tyuratam, NIIP-5, GIK-5), Tyuratam, ..."
1,1957 β (002),03.11.1957,Sputnik 2 (PS-2 #1),Sputnik (1),LC-1/5,,Ba,"Baikonur (Tyuratam, NIIP-5, GIK-5), Tyuratam, ..."
2,1957-F01,06.12.1957,Vanguard (Test Satellite F),Vanguard,LC-18A,Failure,CC,"Cape Canaveral Air Force Station, Eastern Test..."
3,1958 α (001),01.02.1958,Explorer 1,Juno-1,LC-26A,,CC,"Cape Canaveral Air Force Station, Eastern Test..."
4,1958-F01,05.02.1958,Vanguard (Test Satellite G),Vanguard,LC-18A,Failure,CC,"Cape Canaveral Air Force Station, Eastern Test..."
...,...,...,...,...,...,...,...,...
6358,2022-F07,21.12.2022,Pléiades-Neo 5 (VHR-2020 3)Pléiades-Neo 6 (VHR...,Vega-C,ELV,Failed,Ko,"Centre Spatial Guyanais (CSG), Kourou, French ..."
6359,2022-176,27.12.2022,Gaofen 11-04 (GF 11-04),CZ-4B,LC-9,,TY,"Taiyuan Satellite Launch Center (TSLC), Wuzhai..."
6360,2022-177,28.12.2022,Starlink v1.5 G5-1-1 (Starlink 5382)Starlink v...,Falcon-9 v1.2 (Block 5),SLC-40,,CC,"Cape Canaveral Air Force Station, Eastern Test..."
6361,2022-178,29.12.2022,SY 10-02,CZ-3B/G2(2),LC-2,,Xi,"Xichang Space Center (Songlin), Sichuan, China"


In [110]:
df_proye['Configuration:']=df_proye['Configuration:'].str.replace('\n','')
df_proye['Operator:']=df_proye['Operator:'].str.replace('.','')

In [111]:
df_proye.to_csv('csv/misiones',index=False),df_proyectos.to_csv('csv/proyectos',index=False)

(None, None)